In [1]:
import pandas as pd
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode
import os


C:\Users\Admin\AppData\Local\Temp\ipykernel_7892\121653351.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Define the secret key (must be 16, 24, or 32 bytes long)
SECRET_KEY = b'policedata111124'

In [3]:
# Function to encrypt a value using AES
# Function to encrypt a value using AES
def encrypt_value(value):
    cipher = AES.new(SECRET_KEY, AES.MODE_CBC, os.urandom(16))
    ct_bytes = cipher.encrypt(pad(value.encode(), AES.block_size))
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    return iv, ct

In [4]:
# Function to decrypt a value using AES
def decrypt_value(iv, ct):
    iv = b64decode(iv)
    ct = b64decode(ct)
    cipher = AES.new(SECRET_KEY, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode('utf-8')

In [5]:
# Encrypt personal information in a DataFrame
def encrypt_dataframe(df):
    personal_info_columns = []
    encrypted_columns = {}
    
# Identify columns containing personal information
    for column in df.columns:
        if any(keyword in column.lower() for keyword in ['name', 'address', 'phone', 'email','cast']):
            personal_info_columns.append(column)
    print(personal_info_columns)
    # Encrypt personal information in identified columns
    for column in personal_info_columns:
        iv_list, ct_list = [], []
        for value in df[column][:100000]:  
            if pd.notnull(value):  # Check if value is not missing
                iv, ct = encrypt_value(str(value))
                iv_list.append(iv)
                ct_list.append(ct)
            else:
                iv_list.append(None)  # Add None for missing values
                ct_list.append(None)
        encrypted_columns[f"{column}_IV"] = iv_list
        encrypted_columns[f"{column}_CT"] = ct_list
        df.drop(columns=[column], inplace=True)
        
    # Add encrypted columns to DataFrame
    for key, value in encrypted_columns.items():
        df[key] = value

In [6]:
# Decrypt personal information in a DataFrame
def decrypt_dataframe(df):
    for column in df.columns:
        if column.endswith('_CT'):
            iv_column = column.replace('_CT', '_IV')
            decrypted_values = []
            for i in range(len(df)):
                iv = df.at[i, iv_column]
                ct = df.at[i, column]
                if iv is not None and ct is not None:  # Check if value is not missing
                    decrypted_value = decrypt_value(iv, ct)
                else:
                    decrypted_value = None  # Handle missing values
                decrypted_values.append(decrypted_value)
            df[column.replace('_CT', '')] = decrypted_values
            df.drop(columns=[column, iv_column], inplace=True)

In [7]:
def encrypt_csv(input_file, output_file):
    # Read CSV into DataFrame
    df = pd.read_csv(input_file)
    
    # Encrypt personal information in the DataFrame
    encrypt_dataframe(df)
    
    # Write encrypted DataFrame to a new CSV file
    df.to_csv(output_file, index=False)


In [8]:
def decrypt_csv(input_file, output_file):
    # Read encrypted CSV into DataFrame
    df = pd.read_csv(input_file)
    
    # Decrypt personal information in the DataFrame
    decrypt_dataframe(df)
    
    # Write decrypted DataFrame to a new CSV file
    df.to_csv(output_file, index=False)

In [9]:
# Example usage:
input_file = 'AccusedData.csv'
output_file_encrypted = 'encrypted3.csv'
output_file_decrypted = 'decrypted3.csv'


In [10]:
# Encrypt personal information in the CSV file
encrypt_csv(input_file, output_file_encrypted)

# Decrypt personal information in the encrypted CSV file
decrypt_csv(output_file_encrypted, output_file_decrypted)

['District_Name', 'UnitName', 'AccusedName', 'Person_Name', 'Caste', 'PresentAddress', 'PermanentAddress', 'Nationality_Name']


ValueError: Length of values (100000) does not match length of index (3023698)